<a href="https://colab.research.google.com/github/matule00/mmd/blob/main/code/01ZLMA_ex09_Poisson_01_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01ZLMA - Exercise 09
Exercise 09 of the course 01ZLMA.

## Contents

* Log-linear models with Poisson distributed data
* Example from the Lecture 10 (Section 7.7. from lecture notes)



## Poission regression

Poission regression is good for modeling random variables expressing the number of occurs of independent events in given time period.  
It also proves to be more suitable for binomial data if the number of repetitions is large and the probability of success low.


*   $Y_1, \ldots, Y_n$ $iid$ $Y_i \sim Po(\mu_i)$, where $\mu_i = s_i λ_i$ and $s_i$ is known sample size.


*   Canonical link function $g(x) = ln(x)$: $\eta_i = g(\mu_i) = ln(\mu_i) = ln(s_i) + ln(\lambda_i) = ln(s_i) + x_i^T \beta$, where $i= 1,\ldots, n$ and $\mu_i = E[Y_i] = s_i\lambda_i = s_i e^{x_i^T \beta} = e^{ln(s_i) + {x_i^T \beta}}$.

where $ln(si)$ is called an offset.

Average number of events per unit of time/sample size is given by: $λ_i = \frac{E[Y_i]}{s_i}$"



Q: What is the estimated regresion parametr corresponding to offset?

Let us assume exercise 7.7 from lecture notes (Dobson 9.2.1) - British doctors' smoking and coronary death. (https://reneues.files.wordpress.com/2010/01/an-introduction-to-generalized-linear-models-second-edition-dobson.pdf)

Data from the famous doctors study of smoking conducted by Sir Richard Doll and colleagues.

---



In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt


In [ ]:
# Please note that this cell works may not work in other env-s that Google Colab
!pip install wget
import wget
url = "https://github.com/francji1/01ZLMA/raw/main/code/helpers.py"
wget.download(url, '../content/helpers.py')  # path where Colab can find libraries
from helpers import Anova

In [ ]:
!pip install dfply
from dfply import *  # handy module to mimic R dplyr library

In [ ]:
data_poiss_url = "https://raw.githubusercontent.com/francji1/01ZLMA/main/data/smoke.csv"
data_poiss = pd.read_csv(data_poiss_url, sep=';')


In [ ]:
data_poiss

In [ ]:
# Load the R magic extension
%load_ext rpy2.ipython


In [ ]:
%%R -o doctors
install.packages("dobson")
library(dobson)
data(doctors)
? doctors

In [ ]:
doctors

In [ ]:
df = data_poiss.copy()
df['age_avg'] = [40, 50, 60, 70, 80, 40, 50, 60, 70, 80]
df['age_min'] = [35, 45, 55, 65, 75, 35, 45, 55, 65, 75]
df['n_min'] = np.ceil(df['person_years'] / (df['age_min'] - 10))
df['n_avg'] = np.ceil(df['person_years'] / (df['age_avg'] - 10))
df['living'] = df['n_min'] - df['deaths']
df['death_rate'] = df['deaths'] / (df['person_years'] / 10000)

df

Add a death rate: the number of deaths per 10,000 person years:
$$ \text{death_rate} = \frac{deaths}{\frac{\text{person_years}}{10000}}$$


In [ ]:
is_factor_smoke = pd.api.types.is_categorical_dtype(data_poiss['smoke'])
print(is_factor_smoke)

# Convert 'smoke' to a categorical type
data_poiss['smoke'] = data_poiss['smoke'].astype('category')

# Convert 'age_group' to a numeric category type
data_poiss['agecat'] = data_poiss['age_group'].astype('category').cat.codes


In [ ]:
df['agecat'] = data_poiss['agecat']

In [ ]:
df

In [ ]:
data_poiss

## The simplest additive model
We start with the simplest model with the variables `smoke` and `agecat`

In [ ]:
# Define the model formula without including the offset in the formula string
formula = 'deaths ~  smoke + agecat'

# Define the offset
offset = np.log(data_poiss['person_years'])

# Fit the Poisson regression model
mdl = smf.glm(formula=formula, data=data_poiss, offset=offset,
              family=sm.families.Poisson(link=sm.families.links.Log())).fit()

# Print the summary of the model
print(mdl.summary())

In [ ]:

# Calculate the log of person years and add it as a new column
exposure = (data_poiss['person_years'])
# Include the pre-calculated offset column directly in the formula
formula = 'deaths ~ smoke + agecat'

# Fit the Poisson regression model
mdl = smf.glm(formula=formula, data=data_poiss,
              exposure=exposure,  family=sm.families.Poisson()).fit(method='bfgs')

# Print the summary of the model
print(mdl.summary())

Deviation statistics have huge value and the model doesn't fit the data that well.
We show the dependence of the logarithm of the scaled Y values on the variable agecat

In [ ]:
# Calculate the scaled deaths
data_poiss['y_scaled'] = data_poiss['deaths'] / data_poiss['person_years'] * 100000

# Prepare to plot
plt.figure(figsize=(10, 6))

# Plot data: different markers and colors based on the 'smoke' category
for smoke_category in [0, 1]:
    subset = data_poiss[data_poiss['smoke'] == smoke_category]
    plt.scatter(subset['agecat'], np.log(subset['y_scaled']),
                marker='o' if smoke_category == 0 else 's',
                color='red' if smoke_category == 0 else 'blue',
                s=120,  # equivalent to cex in R
                label=f'Smoke={smoke_category}')

# Setting up the plot labels and title
plt.xlabel('Age Category')
plt.ylabel('Log of Deaths per 100,000 Person Years')
plt.title('Log of Scaled Deaths by Age Category and Smoking Status')
plt.legend(title='Smoking Status')
plt.grid(True)

# Show the plot
plt.show()




It is clear that the relationship is not linear, so we will add the variable agecat^2 to the model

## Model with a quadratic dependence on age - Model 0

In [ ]:
# Define the offset
offset = np.log(data_poiss['person_years'])

# Define the model formula to include the quadratic term for 'agecat'
formula_0 = 'deaths ~  smoke + agecat + I(agecat ** 2)'

# Fit the Poisson regression model with the corrected link function (using Log due to deprecation of 'log')
mdl_0 = smf.glm(formula=formula_0, data=data_poiss,  offset=offset,

                family=sm.families.Poisson(link=sm.families.links.Log())).fit()

# Print the summary of the model
print(mdl_0.summary())

All regressors in the model are significant and the value of the deviance statistic has dropped to 12.176; let's compare it with the critical value of the LRT test.

In [ ]:
from scipy.stats import chi2

# Extract the model matrix
Xm = mdl_0.model.exog
n = Xm.shape[0]  # number of rows in the model matrix
p = Xm.shape[1]  # number of parameters in the model

# Print the model matrix, n and p
print("Model Matrix (Xm):\n", Xm)
print("Number of rows (n):", n)
print("Number of parameters (p):", p)

# Calculate critical value using the chi-squared distribution
c_val = chi2.ppf(0.95, df=n-p)
print("Critical value (c_val):", c_val)

# Calculate p-value using the model's deviance and degrees of freedom
p_val = chi2.sf(mdl_0.deviance, df=n-p)  # sf is the survival function, equivalent to 1-cdf
print("P-value (p_val):", p_val)



We did not reject the **hypothesis** that the model describes the data well at the 5% significance level. Let's try adding an interaction to the model (a potential change in dependence on smoking with increasing age was visible in the lecture slide).

## Model with quadratic age dependence and interaction - Model 1

In [ ]:
# Define the model formula to include the interaction term between 'smoke' and 'agecat'
formula = 'deaths ~ smoke + agecat + I(agecat ** 2) + smoke:agecat'

# Fit the Poisson regression model with the Log link function
mdl_1 = smf.glm(formula=formula, data=data_poiss, offset = offset,
                family=sm.families.Poisson(link=sm.families.links.Log())).fit()

# Print the summary of the model
print(mdl_1.summary())


All variables in the model are significant and the deviance statistic value has dropped to 1.6354. We will get the critical value for the LRT test.

In [ ]:
# Extract the model matrix
Xm = mdl_1.model.exog
n = Xm.shape[0]  # number of rows in the model matrix
p = Xm.shape[1]  # number of parameters in the model

# Print the number of observations and parameters
print("Number of observations (n):", n)
print("Number of parameters (p):", p)

# Calculate the critical value using the chi-squared distribution
c_val = chi2.ppf(0.95, df=n-p)
print("Critical value (c_val):", c_val)

# Calculate the p-value using the model's deviance and degrees of freedom
p_val = chi2.sf(mdl_1.deviance, df=n-p)  # sf is the survival function, equivalent to 1-cdf
print("P-value (p_val):", p_val)


We do not reject the hypothesis of model adequacy with a p-value of 0.897. This indicates that the model describes the data well.

For illustration, let's also compute the value of the deviance statistic using the formula from the lecture.


LRT Test for Poison regression by Deviance stistics:
$$S = 2\sum_{i=1}^n \left[ y_i (ln(\frac{y_i}{s_i}) - x_i^T \hat{\beta}) - s_i (\frac{y_i}{s_i} - e^{x_i^T \hat{\beta}}) )\right] = 2\sum_{i=1}^n \left[ y_i ln(\frac{y_i}{\hat{\mu_i}}) - (y_i - \hat{\mu_i})\right]
$$

In [ ]:
# 'deaths' is the observed values
y = df['deaths']

# Calculate the predicted values from the model
mu_hat = mdl_1.predict()

# Calculate deviance for each observation
dev = y * np.log(y / mu_hat) - (y - mu_hat)

# Calculate the sum of deviance, multiplied by 2
dev_stat = 2 * np.sum(dev)
print("Deviance Statistic:", dev_stat)


We get the same value as using the glm() function.

We will also test the significance of the interaction using a formula derived in the lecture.


In [ ]:
# Calculate the predicted values from model 0
mu_tilde = mdl_0.predict()

# Calculate the manual deviance statistic
dev = y * np.log(mu_hat / mu_tilde) - (mu_hat - mu_tilde)
dev_stat = 2 * np.sum(dev)
print("Manual Deviance Statistic:", dev_stat)

# Compute the difference in deviance from the fitted models for verification
deviance_diff = mdl_0.deviance - mdl_1.deviance
print("Deviance Difference from sm.glm: (mdl_0 - mdl_1):", deviance_diff)



for the critical value of the test it holds



In [ ]:
# Calculate the critical value for a chi-squared distribution at the 5% significance level and 1 degree of freedom
c_val = chi2.ppf(0.95, 1)  # Using 0.95 for the upper tail, equivalent to R's lower.tail = FALSE with 0.05
print("Critical value (c_val):", c_val)

# Calculate the p-value for the chi-squared test statistic
p_val = chi2.sf(dev_stat, 1)  # 'sf' is the survival function, equivalent to 1-cdf, and matches R's lower.tail = FALSE
print("P-value (p_val):", p_val)

therefore, the interaction is significant in the model. Alternatively, we can use directly the anova() function,


In [ ]:
anova = Anova()

anova(mdl_0, mdl_1, test='chisq')


which returns the same result."


## Analysis of Model 1


Scatterplot for observed and predicted values of the explained variable


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y, mu_hat, color='red', s=10)  # Scatter plot of observed vs. predicted
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'b--', lw=2)  # Line y=x
plt.xlabel('Observed Deaths')
plt.ylabel('Predicted Deaths')
plt.title('Observed vs. Predicted')
plt.grid(True)
plt.show()


The observed and predicted values correspond very well.

Residual plots


In [ ]:
    Y = mdl_1.model.endog
    mu_hat = mdl_1.fittedvalues
    phi_hat = mdl_1.scale
    var_weights = mdl_1.model.var_weights if mdl_1.model.var_weights is not None else np.ones_like(Y)
    V = mdl_1.family.variance(mu_hat)

    # Compute leverage values
    influence = mdl_1.get_influence(observed=False)  # Adjust as necessary for your model
    h_ii = influence.hat_matrix_diag  # Leverage values

    # Working Residuals
    working_residuals = mdl_1.resid_working

    # Pearson Residuals
    pearson_residuals = (Y - mu_hat) / np.sqrt(var_weights * V)

    # Pearson Standardized Residuals
    pearson_standardized_residuals = pearson_residuals / np.sqrt(phi_hat * (1 - h_ii))

    # Calculate  Pearson residuals from mdl_1
    r_ps = mdl_1.resid_pearson
    # Deviance Residuals
    deviance_residuals = mdl_1.resid_deviance

    # Deviance Standardized Residuals
    deviance_standardized_residuals = deviance_residuals / np.sqrt(phi_hat * (1 - h_ii))

In [ ]:
# Setting up the plot area
fig, axs = plt.subplots(2, 2, figsize=(12, 12))
fig.suptitle('Residual Plots')

# Residuals vs. Fitted Values
axs[0, 0].scatter(mu_hat, r_ps, color='red', s=13)
axs[0, 0].axhline(0, color='black', lw=2)
axs[0, 0].set_title('Residuals vs. Fitted Values')
axs[0, 0].set_xlabel('Fitted Values')
axs[0, 0].set_ylabel('Standardized Residuals')

# Residuals vs. Age Category
axs[0, 1].scatter(data_poiss['agecat'], r_ps, color='red', s=13)
axs[0, 1].axhline(0, color='black', lw=2)
axs[0, 1].set_title('Residuals vs. Age Category')
axs[0, 1].set_xlabel('Age Category')

# Residuals vs. Smoke
axs[1, 0].scatter(data_poiss['smoke'], r_ps, color='red', s=13)
axs[1, 0].axhline(0, color='black', lw=2)
axs[1, 0].set_title('Residuals vs. Smoke')
axs[1, 0].set_xlabel('Smoke')

# QQ Plot of the Residuals
sm.qqplot(deviance_standardized_residuals, line='45', ax=axs[1, 1], color='red')
axs[1, 1].set_title('QQ-plot of Residuals')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


given the small number of observations, no significant problem is visible here.

**Influential observations and leverage points**



In [ ]:
# Cook's distance
c_d = mdl_1.get_influence().cooks_distance[0]

# Leverage values
lev = mdl_1.get_influence().hat_matrix_diag

# Set up the plot area
plt.figure(figsize=(12, 6))

# Plot Cook's distance
plt.subplot(1, 2, 1)
plt.scatter(range(len(c_d)), c_d, color='red', s=13)
plt.xlabel('Index')
plt.ylabel("Cook's Distance")
plt.title("Cook's Distance")
plt.ylim(0, 2.1)

# Add horizontal line at 8/(n-2*p)
plt.axhline(8/(len(mdl_1.model.endog - 2*mdl_1.df_model)), color='blue', linestyle='--')


# Plot leverage values
plt.subplot(1, 2, 2)
plt.scatter(range(len(lev)), lev, color='red', s=13)
plt.xlabel('Index')
plt.ylabel('Leverage')
plt.title('Leverage Values')
plt.ylim(0, 1.2)

# Add horizontal line at 2*p/n for leverage plot
plt.axhline(2 * mdl_1.df_model / len(mdl_1.model.endog), color='blue', linestyle='--')

plt.tight_layout()
plt.show()


In neither case do we see suspiciously large values

**Interpretation of parameters**

Let's calculate the relative risks in Model 1

In [ ]:
# Extract estimated coefficients
par_est_1 = mdl_1.params

# Display estimated coefficients
print("Estimated coefficients:\n", par_est_1)

# Calculate relative risks
RR_1 = np.exp(par_est_1)

# Display relative risks
print("Relative risks:\n", RR_1)

Discussion of the obtained values and calculation of relative risks for individual age categories are provided in Section 7.7 of the lecture materials.

For Model 0 without interactions, the situation is simpler, and it is easy to obtain estimates of RR and corresponding confidence intervals.

In [ ]:
# Extract coefficients and confidence intervals
coefficients = mdl_1.params
conf_intervals = mdl_1.conf_int()

# Calculate exponential of coefficients and confidence intervals
exp_values = np.exp(np.concatenate((coefficients.values.reshape(-1, 1), conf_intervals), axis=1))

# Create a DataFrame
df_exp_values = pd.DataFrame(exp_values, columns=['Estimated Values', 'Lower CI', 'Upper CI'], index=coefficients.index)

# Add parameter names as a separate column
df_exp_values['Parameter Names'] = coefficients.index

# Reorder the columns
df_exp_values = df_exp_values[['Parameter Names', 'Estimated Values', 'Lower CI', 'Upper CI']]

# Display the DataFrame
print(df_exp_values)


In [ ]:
mu_hat

Plots:

In [ ]:
df['y'] = df['deaths']

df['mu_tilde'] = mu_tilde
df['mu_hat'] = mu_hat



In [ ]:
df

In [ ]:
df['y_scaled'] = df['y'] / df['person_years'] * 100000
df['smoke'] = pd.Categorical(df['smoke'], categories=["1", "0"], ordered=True)
df['color'] = df['smoke'].map({"1": "red", "0": "blue"})
df['marker'] = df['smoke'].map({"1": 15, "0": 16})

# Prepare scaled models output
df['mu_tilde_scaled'] = df['mu_tilde'] / df['person_years'] * 100000
df['mu_hat_scaled'] = df['mu_hat'] / df['person_years'] * 100000

# Splitting data for plotting based on smoke status for lines
grouped = df.groupby('smoke')

# Setup the plot with 2 subplots
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

# Plot for Model 0 without interaction
axes[0].scatter(df['agecat'], df['y_scaled'], c=df['color'], marker='o', label=df['smoke'])
for name, group in grouped:
    axes[0].plot(group['agecat'], group['mu_tilde_scaled'], color=group['color'].iloc[0], linewidth=2)
axes[0].set_title('Model 0 without interaction')
axes[0].set_xlabel('Age group')
axes[0].set_ylabel('Deaths per 100,000 person-years')
axes[0].legend(handles=[plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Smokers'),
                        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Non-smokers')], loc='upper left')

# Plot for Model 1 with interaction
axes[1].scatter(df['agecat'], df['y_scaled'], c=df['color'], marker='o', label=df['smoke'])
for name, group in grouped:
    axes[1].plot(group['agecat'], group['mu_hat_scaled'], color=group['color'].iloc[0], linewidth=2)
axes[1].set_title('Model 1 with interaction')
axes[1].set_xlabel('Age group')
axes[1].set_ylabel('Deaths per 100,000 person-years')
axes[1].legend(handles=[plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Smokers'),
                        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Non-smokers')], loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame and it's already defined

# Scale 'y' by 'person_years' and multiply by 100000
df['y_scaled'] = df['y'] / df['person_years'] * 100000

# Setting 'smoke' as a categorical variable with a specific order
df['smoke'] = pd.Categorical(df['smoke'], categories=["1", "0"], ordered=True)

# Map 'smoke' to colors and markers
df['color'] = df['smoke'].map({"1": "red", "0": "blue"})

# Prepare scaled models output
df['mu_tilde_scaled'] = df['mu_tilde'] / df['person_years'] * 100000
df['mu_hat_scaled'] = df['mu_hat'] / df['person_years'] * 100000

# Group data by 'smoke' status for plotting
grouped = df.groupby('smoke')

# Setup the plot with 2 subplots
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

# Plot for Model 0 without interaction
for name, group in grouped:
    axes[0].scatter(group['agecat'], group['y_scaled'], c=group['color'], marker='o', alpha=0.7)
    axes[0].plot(group['agecat'], group['mu_tilde_scaled'], color=group['color'].iloc[0], linewidth=2)

axes[0].set_title('Model 0 without interaction')
axes[0].set_xlabel('Age group')
axes[0].set_ylabel('Deaths per 100,000 person-years')
axes[0].legend(handles=[plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Smokers'),
                        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Non-smokers')], loc='upper left')

# Plot for Model 1 with interaction
for name, group in grouped:
    axes[1].scatter(group['agecat'], group['y_scaled'], c=group['color'], marker='o', alpha=0.7)
    axes[1].plot(group['agecat'], group['mu_hat_scaled'], color=group['color'].iloc[0], linewidth=2)

axes[1].set_title('Model 1 with interaction')
axes[1].set_xlabel('Age group')
axes[1].set_ylabel('Deaths per 100,000 person-years')
axes[1].legend(handles=[plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Smokers'),
                        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Non-smokers')], loc='upper left')

plt.tight_layout()
plt.show()


### Task:
* Try to model with and without offset. Compare results, why to use/ not use offset?
* Try Poisson distribution with factor variables (contingency tables approach).



## Your turn: HW

Exercise 9.2 from
(https://reneues.files.wordpress.com/2010/01/an-introduction-to-generalized-linear-models-second-edition-dobson.pdf)

In the dataframe data(insurance) you have numbers of insurance policies, `n`, and numbers of
claims, `y`, for cars in various insurance categories, `car`, tabulated by age of policy holder, `age`, and district where the policy holder lived `district` (district = 1,for London and other major cities and district = 0, otherwise). The dataset is derived from the CLAIMS data set in Aitkin et al. (1989) obtained from a paper by Baxter, Coutts and Ross (1980).

* Calculate the rate ofclaims y/n for each category and plot the rates by
AGE, CAR and DIST to get an idea ofthe main effects ofthese factors.
* Use Poisson regression to estimate the main effects (each treated as categorical and modelled using indicator variables) and interaction terms.
* Based on the modelling in (b), Aitkin et al. (1989) determined that all the interactions were unimportant and decided that AGE and CAR could be
treated as though they were continuous variables. Fit a model incorporating
these features and compare it with the best model obtained in (b). What
conclusions do you reach?

###Load the dataset

In [ ]:
%%R -o insurance
install.packages("dobson")
library(dobson)

data(insurance)
insurance
? insurance

###Elementary statistics

In [ ]:
insurance.describe()

In [ ]:
dataset = insurance.copy()
#dataset2 = insurance.copy()

dataset['car'] = dataset['car'].astype('category')
dataset['district'] = dataset['district'].astype('category')
dataset['age'] = dataset['age'].astype('category')

#dataset2['district'] = dataset2['district'].astype('category')

dataset.head()

In [ ]:
#is_factor_smoke = pd.api.types.is_categorical_dtype(dataset2['car'])
#print(is_factor_smoke)

###Rate of claims


*   decreasing claim rate by age
*   increasing claim rate by car
*   increasing claim rate by district



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you already have the insurance_df DataFrame loaded

# Calculate the rate of claims (y/n) for each category
dataset['claim_rate'] = dataset['y'] / dataset['n']

# Plot the rates by age
plt.figure(figsize=(10, 6))
sns.barplot(x='age', y='claim_rate', data=dataset)
plt.title('Claim Rate by Age')
plt.xlabel('Age Group')
plt.ylabel('Claim Rate')
plt.show()

# Plot the rates by car
plt.figure(figsize=(10, 6))
sns.barplot(x='car', y='claim_rate', data=dataset)
plt.title('Claim Rate by Car')
plt.xlabel('Car Insurance Category')
plt.ylabel('Claim Rate')
plt.show()

# Plot the rates by district
plt.figure(figsize=(10, 6))
sns.barplot(x='district', y='claim_rate', data=dataset)
plt.title('Claim Rate by District')
plt.xlabel('District')
plt.ylabel('Claim Rate')
plt.show()

###Full model
One can see that the model contains many of insignificant variables.

In [ ]:
import statsmodels.api as sm

# Define the offset
offset = np.log((dataset['y']+1) / (dataset['n']+1))

# Define the model formula
formula1 = 'y ~ age + car + district + age:car + age:district + car:district'

# Fit the Poisson regression model with the Log link function
mdl_f = smf.glm(formula=formula1, data=dataset, offset = offset,
                family=sm.families.Poisson(link=sm.families.links.Log())).fit()

# Print the summary of the model
print(mdl_f.summary())

###Simplier model
This model describes less deviance, while having all variables significant.

In [ ]:
import statsmodels.api as sm

# Define the offset
offset = np.log((dataset['y']+1) / (dataset['n']+1))

# Define the model formula
formula2 = 'y ~ age + car + district'

# Fit the Poisson regression model with the Log link function
mdl_s = smf.glm(formula=formula2, data=dataset, offset = offset,
                family=sm.families.Poisson(link=sm.families.links.Log())).fit()

# Print the summary of the model
print(mdl_s.summary())

###Analysis of simplier model

By the Anova method, one can see that the utilizing the interactions in the full model can be obtained better model. The null hypothesis that the models are equal is rejected with $p$-value of 0,02.

In [ ]:
anova = Anova()

anova(mdl_f, mdl_s, test='chisq')

In [ ]:
y = dataset['y']
mu_hat = mdl_s.predict()

plt.figure(figsize=(7, 5))
plt.scatter(y, mu_hat, color='red', s=10)  # Scatter plot of observed vs. predicted
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'b--', lw=2)  # Line y=x
plt.xlabel('Observed claims')
plt.ylabel('Predicted claims')
plt.title('Observed vs. Predicted')
plt.grid(True)
plt.show()

In [ ]:
def residual_plot(dataset, mod):
    Y = mod.model.endog
    mu_hat = mod.fittedvalues
    phi_hat = mod.scale
    var_weights = mod.model.var_weights if mod.model.var_weights is not None else np.ones_like(Y)
    V = mod.family.variance(mu_hat)

    # Compute leverage values
    influence = mod.get_influence(observed=False)  # Adjust as necessary for your model
    h_ii = influence.hat_matrix_diag  # Leverage values

    # Working Residuals
    working_residuals = mod.resid_working

    # Pearson Residuals
    pearson_residuals = (Y - mu_hat) / np.sqrt(var_weights * V)

    # Pearson Standardized Residuals
    pearson_standardized_residuals = pearson_residuals / np.sqrt(phi_hat * (1 - h_ii))

    # Calculate  Pearson residuals from mdl_1
    r_ps = mod.resid_pearson
    # Deviance Residuals
    deviance_residuals = mod.resid_deviance

    # Deviance Standardized Residuals
    deviance_standardized_residuals = deviance_residuals / np.sqrt(phi_hat * (1 - h_ii))

    # Setting up the plot area
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    fig.suptitle('Residual Plots')

    # Residuals vs. Fitted Values
    axs[0, 0].scatter(mu_hat, r_ps, color='red', s=13)
    axs[0, 0].axhline(0, color='black', lw=2)
    axs[0, 0].set_title('Residuals vs. Fitted Values')
    axs[0, 0].set_xlabel('Fitted Values')
    axs[0, 0].set_ylabel('Standardized Residuals')

    # Residuals vs. Age Category
    axs[0, 1].scatter(dataset['age'], r_ps, color='red', s=13)
    axs[0, 1].axhline(0, color='black', lw=2)
    axs[0, 1].set_title('Residuals vs. Age Category')
    axs[0, 1].set_xlabel('Age Category')

    # Residuals vs. Smoke
    axs[1, 0].scatter(dataset['car'], r_ps, color='red', s=13)
    axs[1, 0].axhline(0, color='black', lw=2)
    axs[1, 0].set_title('Residuals vs. Car Category')
    axs[1, 0].set_xlabel('Car category')

    # QQ Plot of the Residuals
    sm.qqplot(deviance_standardized_residuals, line='45', ax=axs[1, 1], color='red')
    axs[1, 1].set_title('QQ-plot of Residuals')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

def cook_infl(mod):
  # Cook's distance
  c_d = mod.get_influence().cooks_distance[0]

  # Leverage values
  lev = mod.get_influence().hat_matrix_diag

  # Set up the plot area
  plt.figure(figsize=(10, 5))

  # Plot Cook's distance
  plt.subplot(1, 2, 1)
  plt.scatter(range(len(c_d)), c_d, color='red', s=13)
  plt.xlabel('Index')
  plt.ylabel("Cook's Distance")
  plt.title("Cook's Distance")
  plt.ylim(0, 2.1)

  # Add horizontal line at 8/(n-2*p)
  plt.axhline(8/(len(mod.model.endog - 2*mod.df_model)), color='blue', linestyle='--')


  # Plot leverage values
  plt.subplot(1, 2, 2)
  plt.scatter(range(len(lev)), lev, color='red', s=13)
  plt.xlabel('Index')
  plt.ylabel('Leverage')
  plt.title('Leverage Values')
  plt.ylim(0, 1.2)

  # Add horizontal line at 2*p/n for leverage plot
  plt.axhline(2 * mod.df_model / len(mod.model.endog), color='blue', linestyle='--')

  plt.tight_layout()
  plt.show()

In [ ]:
residual_plot(dataset, mdl_s)
cook_infl(mdl_s)

###Model without the 'outlier'

In [ ]:
# Remove the first observation using iloc
dataset_cleaned = dataset.drop(dataset.index[0])

# Define the offset
offset = np.log((dataset_cleaned['y']+1) / (dataset_cleaned['n']+1))

# Fit the Poisson regression model with the Log link function
mdl_n = smf.glm(formula=formula2, data=dataset_cleaned, offset = offset,
                family=sm.families.Poisson(link=sm.families.links.Log())).fit()

# Print the summary of the model
print(mdl_n.summary())

residual_plot(dataset_cleaned, mdl_n)
cook_infl(mdl_n)